# RNN Training - Vader Labels

Training using the labels generated by the Vader Sentiment Analyzer.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import re

In [3]:
!pwd

/Users/sankeerthana/Documents/NTU/YEAR_4/SEM_2/CZ4034/IR-Sentiment-Analysis/IR-Sentiment-Analysis


In [25]:
dataset = pd.read_csv('/Users/sankeerthana/Documents/NTU/YEAR_4/SEM_2/CZ4034/IR-Sentiment-Analysis/IR-Sentiment-Analysis/skincare_dataset/product_reviews_labels.csv')
dataset.head()

,Unnamed: 0,product_id,review,vader_label
0,0,4669755719749,This makes my skin smooth and soft and is ligh...,0
1,1,4669755719749,Love the silky texture. It's very lightweight ...,0
2,2,4669755719749,I’ve been trying to find a moisturizer that wo...,0
3,3,4669755719749,HYRAM made me buy it and I’m on my second bott...,0
4,4,4669755719749,It's a nice moisturizer I personally will use ...,0


In [26]:
dataset.columns

Index(['Unnamed: 0', 'product_id', 'review', 'vader_label'], dtype='object')

In [31]:
dataset.drop(columns='Unnamed: 0', axis=1, inplace=True)
dataset.head()

,product_id,review,vader_label
0,4669755719749,This makes my skin smooth and soft and is ligh...,0
1,4669755719749,Love the silky texture. It's very lightweight ...,0
2,4669755719749,I’ve been trying to find a moisturizer that wo...,0
3,4669755719749,HYRAM made me buy it and I’m on my second bott...,0
4,4669755719749,It's a nice moisturizer I personally will use ...,0


### Data Exploration - Class Balance

In [32]:
dataset['vader_label'].value_counts()

 0    14271
 1      751
-1       18
Name: vader_label, dtype: int64

From the above, we can say that the model is pretty skewed, and favours the neutral class as opposed to positive or negative. This means we might need to relook into the labels to see if they are actually of 'neutral' nature or add some of the crawled reviews that have the label as 1 and -1.

### Data Processing

#### Remving Unwanted Punctuations

In [33]:
def depure_data(data):
    # Remove new line characters
    data = re.sub('\s+', ' ', data)

    # Remove distracting single quotes
    data = re.sub("\'", "", data)
        
    return data

for idx, 

In [11]:
output = []
for idx, row in dataset.iterrows():
    output.append(row['review'].lower())

vader_label = pd.DataFrame(dataset['vader_label'])
dataset.drop(['review'], axis=1)
dataset.drop(['vader_label'], axis=1)

dataset['review'] = output
dataset['vader_label'] = vader_label

dataset.head()

,Unnamed: 0,product_id,review,vader_label
0,0,4669755719749,this makes my skin smooth and soft and is ligh...,0
1,1,4669755719749,love the silky texture. it's very lightweight ...,0
2,2,4669755719749,i’ve been trying to find a moisturizer that wo...,0
3,3,4669755719749,hyram made me buy it and i’m on my second bott...,0
4,4,4669755719749,it's a nice moisturizer i personally will use ...,0


In [22]:
dataset.drop(['Unnamed: 0'], axis=1)
dataset.columns

Index(['Unnamed: 0', 'product_id', 'review', 'vader_label'], dtype='object')

#### Converting text into Lowercase

In [ ]:
for idx, row in dataset.iterrows():
    dataset.at[idx,'review'] = str(row['review']).lower()